In [32]:
from biomart import BiomartServer

import pandas as pd

import os.path
import os
import itertools as it
import numpy as np
import plotly.express as px
from plotly.graph_objects import Figure

### 1 TPM

species_name_dict = {
    "drerio": "zebrafish_gene_name_ct_final.csv",
    "xtropicalis": "xenopus_gene_name_ct_final.csv"
}


In [33]:
def plot_cross_species_spec_category_heatmap(mapping_both, species_1, species_2, group_enhanced=False) -> Figure:
    if not group_enhanced:
        mapping_both.loc[
            mapping_both[f"spec_category_{species_1}"] == 'group enhanced', [f"spec_category_{species_1}"]] = 'enhanced'
        mapping_both.loc[mapping_both[f"spec_category_{species_1}"] == 'cell type enhanced', [
            f"spec_category_{species_1}"]] = 'enhanced'

        mapping_both.loc[
            mapping_both[f"spec_category_{species_2}"] == 'group enhanced', [f"spec_category_{species_2}"]] = 'enhanced'
        mapping_both.loc[mapping_both[f"spec_category_{species_2}"] == 'cell type enhanced', [
            f"spec_category_{species_2}"]] = 'enhanced'

    df = mapping_both.loc[mapping_both[f"{species_2}_homolog_orthology_type"] == 'ortholog_one2one'].groupby(
        [f"spec_category_{species_1}"])[f"spec_category_{species_2}"] \
        .value_counts().to_frame('count').reset_index()

    species1_counts = df.groupby(f"spec_category_{species_1}")['count'].sum()
    species2_counts = df.groupby(f"spec_category_{species_2}")['count'].sum()

    df['Species1_Percentage'] = df.apply(
        lambda row:
        (row['count'] / species1_counts[row[f"spec_category_{species_1}"]]) * 100,
        axis=1)
    df['Species2_Percentage'] = df.apply(
        lambda row:
        (row['count'] / species2_counts[row[f"spec_category_{species_2}"]]) * 100,
        axis=1)
    df['Harmonic_Mean'] = 2 / ((1 / df['Species1_Percentage']) + (1 / df['Species2_Percentage']))

    heatmap_data = df.pivot(columns=f"spec_category_{species_1}",
                            index=f"spec_category_{species_2}",
                            values='Harmonic_Mean')

    heatmap_data = np.round(heatmap_data.astype("float32"), 2)

    if not group_enhanced:

        order = ['cell type enriched',
                 'group enriched',
                 'enhanced',
                 'low cell type specificity',
                 'lowly expressed']
    else:
        order = ['cell type enriched',
                 'group enriched',
                 'cell type enhanced',
                 'group enhanced',
                 'low cell type specificity',
                 'lowly expressed']

    fig = px.imshow(heatmap_data[order].reindex(order), title='One2one orthologs',
                    text_auto=True,
                    width=750, height=600,
                    labels=dict(x=species_1, y=species_2, color="% overlap"))
    fig['layout']['yaxis']['autorange'] = "reversed"
    fig.update_yaxes(autorange=True)

    return heatmap_data, fig



In [34]:

species_1, species_2 = ("drerio", "xtropicalis")

if not os.path.isfile(f"o2o_heatmap_figs/{species_1}_{species_2}_o2o_heatmap_1TPM.pdf"):
    print(f"start {species_1}, {species_2}")
 
    classes_sp1 = pd.read_csv(species_name_dict.get(species_1))
    classes_sp2 = pd.read_csv(species_name_dict.get(species_2))
    
    server = BiomartServer("http://www.ensembl.org/biomart")
    # Select the Ensembl Gene ID dataset
    ensembl = server.datasets[f"{species_1}_gene_ensembl"]
    
    # Define the columns for the query
    columns = ['ensembl_gene_id', 'external_gene_name', f"{species_2}_homolog_ensembl_gene",
               f"{species_2}_homolog_orthology_type", f"{species_2}_homolog_associated_gene_name"]
    
    # Query BioMart for the mappings of orthologs genes
    print(f"total genes from {species_1}: {len(set(classes_sp1.gene))}")
    
    response = ensembl.search({
        'query': classes_sp1.gene.tolist(),
        'attributes': columns,
        'mart_instance': 'ensembl'
    })
    

    mapping_df = pd.read_csv(response.url, sep='\t', header=None, names=columns)
    
    mapping_df = mapping_df.loc[mapping_df[f"{species_2}_homolog_associated_gene_name"].notnull()]
    
    mapping_df = mapping_df.loc[mapping_df[f"{species_2}_homolog_orthology_type"]== 'ortholog_one2one', :]
    
    print(f"total genes from {species_1}: {len(set(classes_sp1.gene))}")
    unique_genes_mapped_sp1 = len(list(set(classes_sp1.gene) & set(mapping_df.external_gene_name)))
    
    print(f"ensembl homology mapped genes from {species_1}: {unique_genes_mapped_sp1}")
    
    mapping_sp1 = pd.merge(mapping_df, classes_sp1, left_on='external_gene_name', right_on='gene', how='inner')
    unique_genes_mapped_sp2 = len(list(set(classes_sp2.gene) & set(mapping_df[f"{species_2}_homolog_associated_gene_name"])))
        
    print(f"total genes from {species_2}: {len(set(classes_sp2.gene))}")
    print(f"ensembl homology mapped genes from {species_2}: {unique_genes_mapped_sp2}")
    
    mapping_both = pd.merge(mapping_sp1, classes_sp2, left_on=f"{species_2}_homolog_associated_gene_name", right_on='gene', how='inner',
                    suffixes=(f"_{species_1}", f"_{species_2}")).drop(columns=['external_gene_name'])

    mapping_both.to_csv(f"./homology_mapped_o2o_{species_1}_{species_2}_1TPM.csv")

    
    heatmap_data, o2o_heatmap = plot_cross_species_spec_category_heatmap(species_1=species_1, 
                                                                         species_2=species_2, 
                                                                         mapping_both=mapping_both, 
                                                                         group_enhanced=True)
    
    o2o_heatmap.write_image(f"./{species_1}_{species_2}_o2o_heatmap_1TPM.pdf")
    
    heatmap_data.to_csv(f"./o2o_heatmap_data_{species_1}_{species_2}_1TPM.csv")
    
    print(f"finished {species_1}, {species_2}")

start drerio, xtropicalis
total genes from drerio: 17330
total genes from drerio: 17330
ensembl homology mapped genes from drerio: 7717
total genes from xtropicalis: 9661
ensembl homology mapped genes from xtropicalis: 6274
finished drerio, xtropicalis


In [35]:
mapping_both

,ensembl_gene_id,xtropicalis_homolog_ensembl_gene,xtropicalis_homolog_orthology_type,xtropicalis_homolog_associated_gene_name,gene_drerio,mean_exp_drerio,min_exp_drerio,max_exp_drerio,max_2nd_drerio,n_exp_drerio,...,enrichment_group_xtropicalis,n_enriched_xtropicalis,max_2nd_or_lim_xtropicalis,exps_enhanced_xtropicalis,n_enhanced_xtropicalis,enhanced_in_xtropicalis,spec_category_xtropicalis,dist_category_xtropicalis,spec_score_xtropicalis,enriched_groups_xtropicalis
0,ENSDARG00000069301,ENSXETG00000042724,ortholog_one2one,tmem177,tmem177,0.183647,0.0,0.6351,0.4179,0,...,NaN,0,0.3802,[],0,NaN,lowly expressed,lowly expressed,0.000000,NaN
1,ENSDARG00000104901,ENSXETG00000021099,ortholog_one2one,ostc,ostc,2.886530,0.0,12.9463,5.4093,39,...,Cement_gland_primordium;Endothelial;Epidermal_...,0,19.8031,[],0,NaN,low cell type specificity,expressed in over 90%,0.000000,NaN
2,ENSDARG00000103391,ENSXETG00000018133,ortholog_one2one,rab9b,rab9b,0.040903,0.0,0.1370,0.1298,0,...,NaN,0,0.3231,[],0,NaN,lowly expressed,lowly expressed,0.000000,NaN
3,ENSDARG00000030463,ENSXETG00000047601,ortholog_one2one,tppp3,tppp3,0.199995,0.0,6.2683,0.9083,1,...,NaN,0,0.1266,[],0,NaN,lowly expressed,lowly expressed,0.000000,NaN
4,ENSDARG00000070447,ENSXETG00000017909,ortholog_one2one,slc39a9,slc39a9,0.092472,0.0,0.6388,0.1795,0,...,Blastula;Cement_gland_primordium;Endoderm;Germ...,0,2.2668,[],0,NaN,low cell type specificity,expressed in over 30%,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5632,ENSDARG00000077677,ENSXETG00000010500,ortholog_one2one,pdgfd,pdgfd,0.011022,0.0,0.0649,0.0499,0,...,NaN,0,0.1142,[],0,NaN,lowly expressed,lowly expressed,0.000000,NaN
5633,ENSDARG00000102333,ENSXETG00000027288,ortholog_one2one,fcf1,fcf1,1.060507,0.0,2.1555,2.1328,18,...,Blastula;Blood;Endoderm;Endothelial;Epidermal_...,0,2.3266,[],0,NaN,low cell type specificity,expressed in over 30%,0.000000,NaN
5634,ENSDARG00000005713,ENSXETG00000025507,ortholog_one2one,ethe1,ethe1,0.088758,0.0,0.6533,0.2048,0,...,Myeloid_progenitors,1,4.3715,[17.486000061035156],1,Myeloid_progenitors,cell type enriched,expressed in less than 30%,6.348849,Myeloid_progenitors
5635,ENSDARG00000008623,ENSXETG00000005742,ortholog_one2one,cops3,cops3,0.569348,0.0,1.3495,0.9682,1,...,Blastula;Blood;Cement_gland_primordium;Endoder...,0,2.5441,[],0,NaN,low cell type specificity,expressed in over 30%,0.000000,NaN


In [36]:
import plotly.io as pio
pio.renderers.default='iframe'
from plotly.graph_objects import Figure
import plotly.express as px

In [37]:
def plot_categories_pie(data, kind='spec_category', title=None) -> Figure:
    """

    :param data: the categories output by hpa_gene_classification
    :param kind: the kind of category to plot, choose between spec_category (default) and dist_category
    :param title: title of the plot, usually the species and dataset name
    :return: a plotly.graph_objects.Figure of Pie plot
    """
    if kind not in ('spec_category', 'dist_category'):
        raise ValueError('kind should be either spec_category or dist_category')

    counts = data[kind].value_counts().to_frame().reset_index().rename(
        columns={kind: "counts", "index": 'gene_category'})
    fig = px.pie(counts, values='counts', names='gene_category', title=title)
    fig.show(renderer='iframe')

    return fig

In [38]:
plot_categories_pie(classes_sp1)

In [39]:
plot_categories_pie(classes_sp1.loc[classes_sp1.gene.isin(mapping_both.gene_drerio),])

In [40]:
plot_categories_pie(classes_sp2)

In [41]:
mapping_both

,ensembl_gene_id,xtropicalis_homolog_ensembl_gene,xtropicalis_homolog_orthology_type,xtropicalis_homolog_associated_gene_name,gene_drerio,mean_exp_drerio,min_exp_drerio,max_exp_drerio,max_2nd_drerio,n_exp_drerio,...,enrichment_group_xtropicalis,n_enriched_xtropicalis,max_2nd_or_lim_xtropicalis,exps_enhanced_xtropicalis,n_enhanced_xtropicalis,enhanced_in_xtropicalis,spec_category_xtropicalis,dist_category_xtropicalis,spec_score_xtropicalis,enriched_groups_xtropicalis
0,ENSDARG00000069301,ENSXETG00000042724,ortholog_one2one,tmem177,tmem177,0.183647,0.0,0.6351,0.4179,0,...,NaN,0,0.3802,[],0,NaN,lowly expressed,lowly expressed,0.000000,NaN
1,ENSDARG00000104901,ENSXETG00000021099,ortholog_one2one,ostc,ostc,2.886530,0.0,12.9463,5.4093,39,...,Cement_gland_primordium;Endothelial;Epidermal_...,0,19.8031,[],0,NaN,low cell type specificity,expressed in over 90%,0.000000,NaN
2,ENSDARG00000103391,ENSXETG00000018133,ortholog_one2one,rab9b,rab9b,0.040903,0.0,0.1370,0.1298,0,...,NaN,0,0.3231,[],0,NaN,lowly expressed,lowly expressed,0.000000,NaN
3,ENSDARG00000030463,ENSXETG00000047601,ortholog_one2one,tppp3,tppp3,0.199995,0.0,6.2683,0.9083,1,...,NaN,0,0.1266,[],0,NaN,lowly expressed,lowly expressed,0.000000,NaN
4,ENSDARG00000070447,ENSXETG00000017909,ortholog_one2one,slc39a9,slc39a9,0.092472,0.0,0.6388,0.1795,0,...,Blastula;Cement_gland_primordium;Endoderm;Germ...,0,2.2668,[],0,NaN,low cell type specificity,expressed in over 30%,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5632,ENSDARG00000077677,ENSXETG00000010500,ortholog_one2one,pdgfd,pdgfd,0.011022,0.0,0.0649,0.0499,0,...,NaN,0,0.1142,[],0,NaN,lowly expressed,lowly expressed,0.000000,NaN
5633,ENSDARG00000102333,ENSXETG00000027288,ortholog_one2one,fcf1,fcf1,1.060507,0.0,2.1555,2.1328,18,...,Blastula;Blood;Endoderm;Endothelial;Epidermal_...,0,2.3266,[],0,NaN,low cell type specificity,expressed in over 30%,0.000000,NaN
5634,ENSDARG00000005713,ENSXETG00000025507,ortholog_one2one,ethe1,ethe1,0.088758,0.0,0.6533,0.2048,0,...,Myeloid_progenitors,1,4.3715,[17.486000061035156],1,Myeloid_progenitors,cell type enriched,expressed in less than 30%,6.348849,Myeloid_progenitors
5635,ENSDARG00000008623,ENSXETG00000005742,ortholog_one2one,cops3,cops3,0.569348,0.0,1.3495,0.9682,1,...,Blastula;Blood;Cement_gland_primordium;Endoder...,0,2.5441,[],0,NaN,low cell type specificity,expressed in over 30%,0.000000,NaN


In [42]:
plot_categories_pie(classes_sp2.loc[classes_sp2.gene.isin(mapping_both.xtropicalis_homolog_associated_gene_name),])

In [43]:
def plot_cross_species_dist_category_heatmap(mapping_both, species_1, species_2) -> Figure:

    df = mapping_both.loc[mapping_both[f"{species_2}_homolog_orthology_type"] == 'ortholog_one2one'].groupby(
        [f"dist_category_{species_1}"])[f"dist_category_{species_2}"] \
        .value_counts().to_frame('count').reset_index()

    species1_counts = df.groupby(f"dist_category_{species_1}")['count'].sum()
    species2_counts = df.groupby(f"dist_category_{species_2}")['count'].sum()

    df['Species1_Percentage'] = df.apply(
        lambda row:
        (row['count'] / species1_counts[row[f"dist_category_{species_1}"]]) * 100,
        axis=1)
    df['Species2_Percentage'] = df.apply(
        lambda row:
        (row['count'] / species2_counts[row[f"dist_category_{species_2}"]]) * 100,
        axis=1)
    df['Harmonic_Mean'] = 2 / ((1 / df['Species1_Percentage']) + (1 / df['Species2_Percentage']))

    heatmap_data = df.pivot(columns=f"dist_category_{species_1}",
                            index=f"dist_category_{species_2}",
                            values='Harmonic_Mean')

    heatmap_data = np.round(heatmap_data.astype("float32"), 2)


  
    order = ['expressed in over 90%',
                 'expressed in over 30%',
                 'expressed in less than 30%',
                 'expressed in single',
                 'lowly expressed']

    fig = px.imshow(heatmap_data[order].reindex(order), title='One2one orthologs',
                    text_auto=True,
                    width=750, height=600,
                    labels=dict(x=species_1, y=species_2, color="% overlap"))
    fig['layout']['yaxis']['autorange'] = "reversed"
    fig.update_yaxes(autorange=True)

    return heatmap_data, fig

In [44]:


mapping_both = pd.read_csv(f"homology_mapped_o2o_{species_1}_{species_2}_1TPM.csv")

heatmap_data, o2o_heatmap = plot_cross_species_dist_category_heatmap(species_1=species_1, species_2=species_2, mapping_both=mapping_both)

o2o_heatmap.write_image(f"{species_1}_{species_2}_o2o_heatmap_dist_1TPM.pdf")

heatmap_data.to_csv(f"o2o_heatmap_data_dist_{species_1}_{species_2}_1TPM.csv")

print(f"finished {species_1}, {species_2}")

finished drerio, xtropicalis
